### Importacoes

In [1]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import date, datetime

### Carregamento de dados, limpeza de colunas e linhas vazias e adequacao do tipo de dado

In [ ]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_.xlsx'))

In [ ]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

In [ ]:
# Limpeza de linhas varias (NaN)
listagem = listagem.dropna()

In [ ]:
listagem['Projeto'] = listagem['Projeto'].astype(int)
listagem['Ano'] = listagem['Ano'].astype(int)
listagem = listagem.rename(columns = {'Situação':'Situacao'})

### Criacao e preenchendo de novas colunas

In [ ]:
'''Criacao das colunas: 
codigo do tipo
id do projeto
data de proposicao do projeto
status da tramitacao
conferência da situacao
'''
listagem['Codigo_Tipo'] = ''
listagem['Id_Projeto'] = ''
listagem['Data_Projeto'] = ''
listagem['Tramitacao_Encerrada'] = ''
listagem['Aprovado'] = '' 

#listagem['Tramitacao'] = '' # UNIAO DOS DOIS CAMPOS ANTERIORES EM REVISAO

In [ ]:
# Preenchimento do campo Codigo_Tipo
unknown_count = 0

for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Codigo_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Codigo_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Codigo_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Codigo_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Codigo_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Codigo_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Codigo_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Codigo_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Codigo_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Codigo_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Codigo_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Codigo_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Codigo_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Codigo_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Codigo_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Codigo_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Codigo_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Codigo_Tipo'][i] = 'RCPI'
    elif(listagem["Tipo"][i] == "Pedido de Informação FiscalizaJF"):
        listagem['Codigo_Tipo'][i] = 'PEDFF'
    elif(listagem["Tipo"][i] == "Emenda"):
        listagem['Codigo_Tipo'][i] = 'EMENDA'
    else:
        listagem['Codigo_Tipo'][i] = 'DESCONHECIDO'
        unknown_count += 1

print(f'Fim do processo. Traducao realizada com {unknown_count} registros nao encontrados')

### Importacoes

In [ ]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 3
notfound_count = 0
req_count = 0
max_req = 4900

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                id_projeto = ''
                failed = False
                # Variaveis de requisicao
                num_proj = int(listagem['Projeto'][i])
                ano_proj = int(listagem['Ano'][i])
                tipo_proj = listagem['Codigo_Tipo'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                    #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    req_count+=1
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        try:
                            retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                            id_projeto = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                        except:
                            id_projeto = 'Nao Encontrado'
                            notfound_count += 1
                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)

                listagem['Id_Projeto'][i] = id_projeto
                #resposta.close()
                session.close()
                time.sleep(0.3)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas ou houve mais de 5 mil requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break

    if failed_req > max_failed_req or req_count > max_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

now = datetime.now()
dthr_finalizado = now.strftime("%d/%m/%Y %H:%M:%S")

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}.')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}.')

In [ ]:
# Preenchimento do campo Data_Projeto, Tramitacao_Encerrada e Aprovado
count = 1
failed_req= 0
failed = False
max_failed_req = 5
req_count = 0
max_req = 4900

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto'][i] != '': 
            with requests.Session() as session:
                session.
                data_projeto = ''
                tramitacao_encerrada = ''
                aprovado = ''

                failed = False
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                    #Identificacao da data de registro do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                    except:
                        data_projeto = 'Data nao encontrada'
                        notfound_count_dt += 1


                    #Identificacao do encerramento da tramitacao do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                        if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                            tramitacao_encerrada = 'SIM'
                        else:
                            tramitacao_encerrada = 'NAO'
                    except:
                        tramitacao_encerrada = 'Informacao nao encontrada'
                        notfound_count_tram+=1


                    #Identificacao da aprovacao do projeto
                    if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jurídica' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                    elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetado' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                    elif tramitacao_encerrada == 'NAO': 
                        aprovado = 'TRAMITACAO'
                    #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                    #    if listagem['Situacao'][i] == 'Arquivada': 
                    #        aprovado = 'NAO'        
                    #    else:        
                    #        aprovado = 'SIM'
                    #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                    else:
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                            if(re.search('APROVAD', retorno)):
                                aprovado = 'SIM'
                            else:
                                aprovado = 'NAO'
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1

                except:
                    failed = True
                    time.sleep(180)

                listagem['Data_Projeto'][i] = data_projeto
                listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
                listagem['Aprovado'][i] = aprovado

                resposta.close()
                session.close()
                time.sleep(0.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req or req_count > max_req:
            break
            
    if failed_req > max_failed_req or req_count > max_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Tramitacao_Encerrada'][i] == '' and listagem['Aprovado'][i] == '' and listagem['Id_Projeto'][i] != '':
            count +=1

now = datetime.now()
dthr_finalizado = now.strftime("%d/%m/%Y %H:%M:%S")

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}.')
elif req_count > max_req:
    print(f'Processo finalizado em 5 mil requisicoes. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}.')
else:
    print(f'Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}.')

### Exportando informacoes sobre projetos e autoria para arquivos CSV

In [ ]:
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Codigo_Tipo'
               , 'Id_Projeto', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']
              #, 'Autor'] 
    
if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

In [ ]:
# Logica de identificador de autor (Id_Autor) e identificacao de autoria, separando casos de 'multiautoria'
autoria_df_estrutura = {'nome_autor': [''],'id_projeto': ['']}
autoria_df = pd.DataFrame(columns=['nome_autor','id_projeto'])
autores = ['']
aut = ['']

for i in listagem.index:
    aut = listagem['Autor'][i].split(', ')
    
    
    for item in aut:
        autoria = pd.DataFrame({'nome_autor': item, 
                      'id_projeto': listagem['Id_Projeto'][i]}, index=[0]) 
        autoria_df = autoria_df.append(autoria, ignore_index=True, verify_integrity=False, sort=None) 

# Exportacao da listagem de autorias para arquivo csv
path_to_file = '../ListagemExp/AutoriasExport.csv'
path = Path(path_to_file)
exp_columns = ["id_projeto", "nome_autor"]

if path.is_file():
    exp_header = False
else:    
    exp_header = True

autoria_df.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

# Visualizacao do dataframe e testes

In [15]:
display(listagem)

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,4279,2016,Mensagem do Executivo (Projeto de Lei Compleme...,Institui o Código Sanitário Municipal de Juiz ...,Transformado em Norma Jurídica,Executivo,MSGPC,58080,08/12/2016,SIM,SIM
1,4278,2016,Mensagem do Executivo (Projeto de Lei),Autoriza o dispêndio de transferências corrent...,Transformado em Norma Jurídica,Executivo,MSGPL,58076,06/12/2016,SIM,SIM
2,4277,2016,Mensagem do Executivo (Projeto de Lei),Autoriza a Concessão de Subvenções de Caráter ...,Transformado em Norma Jurídica,Executivo,MSGPL,58077,06/12/2016,SIM,SIM
3,4276,2016,Mensagem do Executivo (Projeto de Lei Compleme...,"Acrescenta os §§ 5º e 6º ao art. 1º, da Lei Co...",Transformado em Norma Jurídica,Executivo,MSGPC,58062,05/12/2016,SIM,SIM
4,4275,2016,Mensagem do Executivo (Projeto de Lei),"Altera dispositivos da Lei nº 10.630, de 30 de...",Arquivada,Executivo,MSGPL,57983,30/11/2016,SIM,NAO
...,...,...,...,...,...,...,...,...,...,...,...
58647,1,2022,Representação,Representação à Polícia Militar de Minas Gerai...,Aprovado,Vagner de Oliveira,PLEI,,,,
58648,1,2022,Pedido de Informação,a quem pertence o imóvel que funcionava o Post...,Aprovado,Cida Oliveira,PEDIF,,,,
58649,1,2022,Requerimento,Audiência Pública para discutirmos a situação ...,Aprovado,Cida Oliveira,REQ,,,,
58650,1,2022,Moção,Moção de Aplauso ao Senhor Marcos de Lima Schu...,Aprovado,João Wagner,MOC,,,,


In [ ]:
'''
  
# Contagem e conferencia de qual eh o ultimo projeto com id_projeto preenchido 
count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "":
        print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])
        print(count)
        break
    else:
        count +=1
'''

In [ ]:
'''
# Conferencia de valores
i = 936

print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], 
      listagem['Id_Projeto'][i], listagem['Data_Projeto'][i], listagem['Situacao'][i])

#https://www.camarajf.mg.gov.br/sal/proposicao.php?num=45773
'''

In [ ]:
'''
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport.csv'
path = Path(path_to_file)

#exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Codigo_Tipo', 
#               'Id_Projeto', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header) 
'''

### Criacao de 'checkpoint' e recuperacao de trabalho parado - especialmente em caso de endpoint fora do ar

In [16]:
# Exportacao da listagem no estado atual para arquivo csv
today = date.today()
file = 'ListagemExport2016-2022_checkpoint_' + today.strftime("%Y-%m-%d")
path_to_file = '../ListagemExp/'+file+'.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
               'Id_Projeto', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 
print(path)

..\ListagemExp\ListagemExport2016-2022_checkpoint_2022-10-12.csv


In [2]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = '..//ListagemExp/ListagemExport2016-2022_checkpoint_2022-10-11.csv'
listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Id_Projeto'] = listagem['Id_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

### Identificacao do estado da importacao dos dados

In [17]:
tem=0
naotem=0

#oid = ''
for i in listagem.index:
    if listagem['Id_Projeto'][i]!='':
#    if listagem['Data_Projeto'][i]!='':
        tem+=1
    else:
        naotem+=1
        #oid=i
print(tem)
print(naotem)
#print(oid)

25692
32960
